In [5]:
import csv

# Pfade der Eingabe- und Ausgabedatei
input_file = 'v0_unique_separat_speeches_labeled.csv'  # Ersetze dies durch den tatsächlichen Dateipfad
output_file = 'test.csv'

# Erwartete Anzahl der Felder
expected_fields = 9

# Datei Zeile für Zeile lesen und überprüfen
with open(input_file, 'r', encoding='utf-8') as infile, open(output_file, 'w', encoding='utf-8', newline='') as outfile:
    reader = csv.reader(infile)
    writer = csv.writer(outfile)

    for row in reader:
        # Überprüfen, ob die Anzahl der Felder stimmt
        if len(row) == expected_fields:
            writer.writerow(row)
        else:
            print(f"Übersprungene Zeile: {reader.line_num} - {len(row)} Felder")

print(f"Bereinigte Datei wurde gespeichert: {output_file}")


Übersprungene Zeile: 2639 - 80 Felder
Übersprungene Zeile: 22437 - 131 Felder
Bereinigte Datei wurde gespeichert: test.csv


In [6]:
import pandas as pd
import numpy as np

# Datei einlesen
def load_data(file_path):
    try:
        df = pd.read_csv(file_path)
        print("Daten erfolgreich geladen.")
        return df
    except Exception as e:
        print(f"Fehler beim Laden der Datei: {e}")
        return None

# Überprüfung der Datenqualität
def check_data_quality(df):
    if df is None:
        print("Keine Daten zum Überprüfen.")
        return

    print("\n--- Grundlegende Statistiken ---")
    print(df.info())
    print("\nErste Zeilen:")
    print(df.head())

    # Vollständigkeit prüfen
    print("\n--- Fehlende Werte ---")
    missing_values = df.isnull().sum()
    print(missing_values)

    # Eindeutigkeit prüfen
    print("\n--- Eindeutige IDs ---")
    duplicate_ids = df[df['id'].duplicated()]
    print(f"Duplikate gefunden: {len(duplicate_ids)}")

    # Konsistenz prüfen (z. B. Datum, Partei)
    print("\n--- Konsistenz der Datumsangaben ---")
    try:
        df['datum'] = pd.to_datetime(df['datum'], errors='coerce')
        invalid_dates = df[df['datum'].isnull()]
        print(f"Ungültige Datumsangaben: {len(invalid_dates)}")
    except Exception as e:
        print(f"Fehler beim Prüfen der Datumsangaben: {e}")

    print("\n--- Konsistenz der Parteiangaben ---")
    print(df['partei'].value_counts())

    # Textqualität prüfen
    print("\n--- Analyse der Redetexte ---")
    df['text_length'] = df['text'].apply(lambda x: len(str(x)) if pd.notnull(x) else 0)
    print(f"Minimale Textlänge: {df['text_length'].min()}")
    print(f"Maximale Textlänge: {df['text_length'].max()}")
    print(f"Durchschnittliche Textlänge: {df['text_length'].mean():.2f}")

    short_texts = df[df['text_length'] < 50]  # Beispielgrenze für zu kurze Texte
    print(f"Zu kurze Texte (Länge < 50 Zeichen): {len(short_texts)}")

    # Label-Qualität prüfen
    print("\n--- Konsistenz der Labels ---")
    print(df['label'].value_counts())

    if 'alt_labels' in df.columns:
        inconsistent_labels = df[df['alt_labels'].isnull() & df['label'].notnull()]
        print(f"Einträge mit fehlenden alternativen Labels: {len(inconsistent_labels)}")

    print("\n--- Prüfung abgeschlossen ---")
    return {
        'missing_values': missing_values,
        'duplicate_ids': duplicate_ids,
        'invalid_dates': invalid_dates,
        'short_texts': short_texts,
    }

# Beispielausführung
def main():
    file_path = "test.csv"  # Pfad zur CSV-Datei
    df = load_data(file_path)
    results = check_data_quality(df)

if __name__ == "__main__":
    main()


Daten erfolgreich geladen.

--- Grundlegende Statistiken ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24279 entries, 0 to 24278
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             24279 non-null  int64 
 1   sitzungs_id    24279 non-null  int64 
 2   dokumenten_id  24279 non-null  object
 3   datum          24279 non-null  object
 4   redner         24279 non-null  object
 5   partei         24059 non-null  object
 6   label          24279 non-null  object
 7   alt_labels     24279 non-null  object
 8   text           24279 non-null  object
dtypes: int64(2), object(7)
memory usage: 1.7+ MB
None

Erste Zeilen:
   id  sitzungs_id dokumenten_id       datum             redner     partei  \
0  50         5267        19/110  2019-09-10   Albert Stegemann    CDU/CSU   
1  48         5267        19/110  2019-09-10  Amira Mohamed Ali  DIE LINKE   
2   2         5267        19/110  2019-09-10       And